In [1]:
import boto3
import json
import pandas as pd

In [60]:
# s3://echonest-intermediate-json/json/AA/SOAAABI12A8C13615F.json
bucket = 'echonest-intermediate-json'
file_name = 'json/AA/SOAAABI12A8C13615F.json'
prefix = 'json/AA'

In [100]:
combined_df = pd.DataFrame(
    {
        "song_id": pd.Series(dtype="str"),
        "spotify_id": pd.Series(dtype="str")
    }
)

In [73]:
s3 = boto3.resource('s3')



In [66]:
combined_df

,song_id,spotify_id
0,SOAAABI12A8C13615F,spotify:track:6i9aLGfG1Id1oCcpgmRjmE
1,SOAAABI12A8C13615F,spotify:track:6i9aLGfG1Id1oCcpgmRjmE


In [82]:
s3Client = boto3.client('s3')

response = s3Client.list_objects_v2(
    Bucket=bucket,
    Prefix=prefix
)

files = [file['Key'] for file in response['Contents']]

In [101]:
for file_name in files:

    content_object = s3.Object(bucket, file_name)
    file_content = content_object.get()['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    
    spotify_id = None

    if len(json_content['response']['songs']) > 0:
        
        track_info = json_content['response']['songs'][0]['tracks']
    
        song_id = json_content['response']['songs'][0]['id']
    
        for track in track_info:
            if track['catalog'] == 'spotify':
                spotify_id = track['foreign_id']
                break
    else:
        song_id = file_name.split('/')[-1].split('.')[0]

    song_df = pd.DataFrame({'song_id': song_id, 'spotify_id': spotify_id}, index=[0])

    combined_df = pd.concat([combined_df, song_df], axis=0).reset_index(drop=True) 


In [113]:
import json
 
data = json.load(open('sample-event.json'))

In [115]:
body = json.loads(data['Records'][0]['body'])